# 3. Tick Rule

A way to determine the buying and selling aggressors when only basic tick data is available is the tick rule.  It's simple and fast: if the current price is above the previous price, then it's an aggressive buyer, if current price is below the previous price, then it's an aggressive seller, if the current price is the same as the previous, then base the aggressor from the previous.

Because the leading data won't have a previous price, the direction can't be known until a price change occurs.

In [ ]:
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import arrow

In [ ]:
tick_directory = "./data/tick"
filename = f"{tick_directory}/NQ_09_23.20230818.parquet"

(pl.scan_parquet(filename)
    .fetch(5)
)

## Use the tick rule to determine the trade direction

In [ ]:
lf = (pl.scan_parquet(filename)
    # sort the data
    .sort("Timestamp")
    # use the when function to determine the direction.
    .with_columns(
        # if the current price > previous price, then set the "tick direction" column
        # to 1.  If the current price < previous price, the set the "tick direction" column
        # to -1. Otherwise, set the value to None.
        pl.when(pl.col("Price") > pl.col("Price").shift())
            .then(pl.lit(1))
            .when(pl.col("Price") < pl.col("Price").shift())
            .then(pl.lit(-1))
        
            # if the prices are equal, then fill in with "None" or null
            .otherwise(pl.lit(None))
        
            .alias("tick direction")
    )
    # use the fill_null function to fill in the null values using the forward (fill) strategy
    .with_columns(
       pl.col("tick direction").fill_null(strategy="forward")
    )
    # multiply the volume column by the tick direction column to get the bid/ask volume
    .with_columns(
        (pl.col("Volume") * pl.col("tick direction")).alias("tick direction")
    )
    # the first few rows will have a null tick direction value, so we'll drop these
    .drop_nulls()
)

In [ ]:
lf.fetch(10)

## Use group_dynamic() to create a time based chart

In [ ]:
interval="5m"
# using the time interval bars.
lf2 = (lf
    # first, limit what data we want to look at
    .filter(
        pl.col("Timestamp").is_between(
            arrow.get("2023-08-18 08:00").datetime.replace(tzinfo=None),
            arrow.get("2023-08-18 16:15").datetime.replace(tzinfo=None)
        )
    )
    .sort("Timestamp")
    # resample the data
    .groupby_dynamic("Timestamp", every=interval)
    .agg(
        pl.col("Price").first().alias("Open"),
        pl.col("Price").max().alias("High"),
        pl.col("Price").min().alias("Low"),
        pl.col("Price").last().alias("Close"),
        pl.col("Volume").sum().alias("Volume"),
        
        # the "tick direction" column contains the +/- volume
        # so summing this will get the delta
        pl.col("tick direction").sum().alias("Delta")
    )
    
    # then use the cumsum() function to keep a running total the delta.
    # i.e. cumulative delta
    .with_columns(
        pl.col("Delta").cumsum().alias("Cumulative Delta")
    )
)

In [ ]:
df = lf2.collect()

## Plot the data

In [ ]:
# get the date from the data.
# we'll use this on the plots.
chart_date = arrow.get(df.row(1,named=True).get("Timestamp"))

In [ ]:
fig = go.Figure()

candlestick_trace = go.Candlestick(
            x=df["Timestamp"],
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close']
        )

fig.add_trace(candlestick_trace)

fig.update_layout(
    title=f"NQ - {chart_date.format('YYYY-MM-DD')}",
    height=600,
    width=1000,
    template="plotly_dark",
    xaxis_rangeslider_visible=False,
    showlegend=False
)

fig.show()   

## Overlay the cumulative delta on the price chart

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

candlestick_trace = go.Candlestick(
            x=df["Timestamp"],
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close']
        )

fig.add_trace(candlestick_trace, secondary_y=False)

# create the markers dictionary for cumulative delta
# scale the volume for the marker size
markers = dict(
    size = [int(x*.001) for x in df.get_column("Volume")],
    color = ["cyan"]* df.get_column("Volume").shape[0]
)
cumulative_delta_Trace = go.Scatter(
        x=df['Timestamp'],
        y=df['Cumulative Delta'],
        name='Cumulative Delta',
        mode="markers",
        marker=markers
    )

fig.add_trace(cumulative_delta_Trace, secondary_y=True)

fig.update_layout(
    title=f"NQ / Cumulative Delta - {chart_date.format('YYYY-MM-DD')}",
    height=600,
    width=1000,
    template="plotly_dark",
    xaxis_rangeslider_visible=False,
    showlegend=False
)

fig.show()   
